# Transfer Matrix and Lattice Dilatation Operator for High-Quality Fixed Points in Tensor Network Renormalization Group
## Reproduction of some of the paper results

This notebook reproduces some of the results presented in the paper "Transfer Matrix and Lattice Dilatation Operator for High-Quality Fixed Points in Tensor Network Renormalization Group." Let us start with loading source codes and ensuring the correct environment is used.

In [6]:
using Pkg
Pkg.activate(".")
include("Tools.jl")
include("KrylovTechnical.jl")
include("GaugeFixing.jl");

  Activating project at `~/Codes/tmp2/GILT_TNR_R`


We will reproduce results from Tables 4 and 5 for the rotating algorithm. First, we need to obtain the fixed point tensor. We begin by finding an initial approximation using the shooting method.

In [2]:
run(`julia --project Lab/critical_temperature.jl --rotate true`)

The latest version of Julia in the `release` channel is 1.10.5+0.x64.linux.gnu. You currently have `1.10.4+0.x64.linux.gnu` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.5+0.x64.linux.gnu and update the `release` channel to that version.
[ Info: Sanity check passed
[ Info: Compute phase for relT=1.0
[ Info: It is LOW_TEMPERATURE
[ Info: gap=0.010000000000000009
[ Info: Compute phase for relT=1.005
[ Info: It is HIGH_TEMPERATURE
[ Info: gap=0.004999999999999893
[ Info: Compute phase for relT=1.0025
[ Info: It is HIGH_TEMPERATURE
[ Info: gap=0.0024999999999999467
[ Info: Compute phase for relT=1.00125
[ Info: It is HIGH_TEMPERATURE
[ Info: gap=0.0012499999999999734
[ Info: Compute phase for relT=1.0006249999999999
[ Info: It is HIGH_TEMPERATURE
[ Info: gap=0.0006249999999998757
[ Info: Compute phase for relT=1.0003125
[ Info: It is HIGH_TEMPERATURE
[ Info: gap=0.00031249999999993783
[ Info: Compute phase for relT=1.0001562499999999
[ Info: It is HIGH_TEM

1362.914602 seconds (1.68 M allocations: 117.834 MiB, 0.02% gc time, 0.12% compilation time: 9% of which was recompilation)


[ Info: It is HIGH_TEMPERATURE
[ Info: gap=7.450595695956963e-11


1.0:1.000013131201267


[ Info: No record in the database, will compute the trajectory from scratch


Process(`julia --project Lab/critical_temperature.jl --rotate true`, ProcessExited(0))

We now apply the Newton algorithm to obtain a more accurate approximate fixed point.

In [3]:
run(`julia --project --threads 40 Lab/newton.jl`)

The latest version of Julia in the `release` channel is 1.10.5+0.x64.linux.gnu. You currently have `1.10.4+0.x64.linux.gnu` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.5+0.x64.linux.gnu and update the `release` channel to that version.


229.523367 seconds (13.80 M allocations: 14.312 GiB, 0.66% gc time, 5.92% compilation time: <1% of which was recompilation)
EIGENVALUES (INITIAL):
1.999776676959205 + 0.0im
-1.0006381029944171 + 0.0im
-0.9984754866505865 + 0.0im
0.773984159175744 + 0.0im
0.6235939365424155 + 0.0im
0.152882071839461 + 0.485048985611728im
0.152882071839461 - 0.485048985611728im
-0.07759411412949238 + 0.4988476529032618im
-0.07759411412949238 - 0.4988476529032618im
Newton step size: 0.002646109766590412
Newton step size: 0.00010968878865217572
Newton step size: 0.00020999149639704427
Newton step size: 0.00012209768857244672
Newton step size: 6.122548774422856e-5
Newton step size: 2.425300838180461e-5
Newton step size: 1.6374254037270193e-5
Newton step size: 9.51143868909394e-6
Newton step size: 5.805493219181222e-6
Newton step size: 3.005322086270694e-6
Newton step size: 1.7404041986947397e-6
Newton step size: 1.0136061338886858e-6
Newton step size: 5.864802216349703e-7
Newton step size: 3.262704820866658

Process(`julia --project --threads 40 Lab/newton.jl`, ProcessExited(0))

Using the fixed point tensor we obtained, we now proceed with the transfer matrix (TM) computation. We begin by loading the tensor, properly normalizing it, and converting it to a simple array. This last step enables us to utilize the @tensor macro.

In [7]:
newton_res=deserialize("newton/rotate=true_30_6.0e-6_1.0e-10_jac_approximation_rank=9.data");
A=newton_res["A_newton"][end] |> ju_to_py;

gilt_pars = Dict(
	"gilt_eps" => 6e-6,
	"cg_chis" => collect(1:30),
	"cg_eps" => 1e-10,
	"verbosity" => 0,
	"bond_repetitions" => 2,
	"recursion_depth" => newton_res["recursion_depth"],
	"rotate" => true,
)
Atmp, lf, _ = py"gilttnr_step"(A, 0.0, gilt_pars);
g = (Atmp.norm())^(-1 / 3)
A = A * g;
A = A.to_ndarray();


Here we define the parity function that enables us to infer $\mathbb{Z}_2$ quantum numbers for tensors, matrices, and vectors. We also define the twist operator and some auxiliary vectors.

In [16]:
using TensorOperations

function parity(tens::Array{T, 4}, Z) where {T}
	@tensor tens_flipped[-1, -2, -3, -4] := tens[1, 2, 3, 4] * Z[1, -1] * Z[2, -2] * Z[3, -3] * Z[4, -4]
	return dot(tens, tens_flipped) / (norm(tens))^2
end

function parity(mat::Matrix{T}, Z) where {T}
	mat_flipped = Z * mat * Z
	return dot(mat, mat_flipped) / (norm(mat))^2
end

function parity(v::Vector{T}, Z) where {T}
	v_flipped = Z * v
	return dot(v, v_flipped) / (norm(v))^2
end

function ⊗(a, b)
	kron(a, b)
end


Z = diagm(vcat(ones(15), -ones(15)));
ZZ = reshape(Z ⊗ Z, 900, 900);
ZZZ = reshape((Z ⊗ Z) ⊗ Z, 900 * 30, 900 * 30)

id = diagm(ones(30))

@tensor twist[-1, -2, -3, -4] := id[-1, -4] * id[-2, -3];
twist = reshape(twist, 900, 900);


We can now proceed with the TM computation. We begin with the direct TM.

In [10]:
@tensor T[-1, -2, -3, -4] := A[1, -3, 2, -1] * A[2, -4, 1, -2];

T = reshape(T, 900, 900);

vals, vecs = eigen(T, sortby = x -> -abs(x));

shifted_dimensions = -log.(vals) ./ pi;

c = -shifted_dimensions[1] * 12;
dimensions_re = real.(shifted_dimensions .- shifted_dimensions[1]);
dimensions_im = imag.(shifted_dimensions .- shifted_dimensions[1]);


println("CENTRAL CHARGE: ", c)
println("-------------------------------")
println("SCALING DIMENSIONS (RE):")

for i ∈ 2:50
	println(dimensions_re[i])
end
println("-------------------------------")

println("SCALING DIMENSIONS (IM):")

for i ∈ 2:50
	println(dimensions_im[i])
end
println("-------------------------------")

println("Z2 EIGENVALUES:")

for i ∈ 2:50
	println(real(parity(vecs[:, i], ZZ)))
end
println("-------------------------------")

println("SPIN MOD 2")

for i ∈ 2:50
	println(-(real.(parity(vecs[:, i], twist)) .- 1) ./ 2)
end


CENTRAL CHARGE: 0.4997450476022938 + 0.0im
-------------------------------
SCALING DIMENSIONS (RE):
0.12498465077784235
1.002010517504987
1.1280823504589697
1.1281431759782727
2.0069940473040955
2.0092808545007776
2.012600438067811
2.0132969470673343
2.1299342289365515
2.1365083499897337
2.149278745558596
2.999449867530442
3.0001033857600725
3.004233600669529
3.027356940521822
3.032795848661699
3.0688969963509276
3.0791119153147664
3.139212798045889
3.1397490302147775
3.150672370236658
3.159066296468325
3.592855561176288
3.6104737433749263
3.6624406929854474
3.7410422556054668
3.7760908928896137
3.7821646618920277
3.7884831857292265
3.8076145261065926
3.8342018528285062
3.8762157052788444
3.883129495117732
3.9608420682237115
3.986437689071991
3.9994928243674965
4.0344658524377275
4.039672433228345
4.043596989925225
4.0602814339433895
4.062903336719354
4.093985820619445
4.098978299131352
4.113562082848104
4.119787922646191
4.122415037804252
4.153463977972903
4.177410882788208
4.18624169

Now, let us repeat the crossed TM computation.

In [11]:
using KrylovKit

@tensor T[-1, -2, -3, -4, -5, -6] := A[-1, -5, 1, -2] * A[1, -6, -4, -3];

T = reshape(T, 900 * 30, 900 * 30);


res = eigsolve(T, 900 * 30, 50; krylovdim = 60);

vals, vecs = res[1], res[2];

shifted_dimensions = -real.(log.(vals)) .* (5 / (4 * pi));

spins = imag.(log.((vals))) * (5 / (2 * pi));


c = -shifted_dimensions[1] * 12;

dimensions = shifted_dimensions .- shifted_dimensions[1];

println("CENTRAL CHARGE: ", c)
println("-------------------------------")
println("SCALING DIMENSIONS :")

for i ∈ 2:50
	println(dimensions[i])
end
println("-------------------------------")
println("SPINDS mod 5:")

for i ∈ 2:50
	println(spins[i])
end

println("-------------------------------")
println("Z2 EIGENVALUES:")
for i ∈ 2:50
	println(real(parity(vecs[i], ZZZ)))
end

CENTRAL CHARGE: 0.5000656473345877
-------------------------------
SCALING DIMENSIONS :
0.1250391803215657
1.000858202794739
1.1245417958140556
1.1245417958140556
1.9982786178176426
1.9982786178176426
2.0017499038645314
2.0017499038645314
2.1245856654827517
2.1245856654827517
2.128877491753982
3.0078401102439667
3.0096684507700613
3.0096684507700613
3.0150426605573593
3.0150426605573593
3.1250689907646705
3.1250689907646705
3.1338161746740845
3.1338161746740845
3.1840677329073612
3.1840677329073612
3.991131459893168
3.991131459893168
4.005097590299604
4.005097590299604
4.0162381679166135
4.026076622681248
4.026076622681248
4.11282594731019
4.11282594731019
4.139924314131775
4.139924314131775
4.148580609658831
4.148580609658831
4.173356221701005
4.173356221701005
4.174928579050616
4.174928579050616
4.1839510758901
4.1839510758901
4.214894780161722
4.214894780161722
4.229231915269486
4.270939882352382
4.287707778760732
4.287707778760732
4.29435655396085
4.3093760067836815
---------------

We now proceed to the Lattice Dilatation Operator (LDO) computation. First, we need to save our critical tensor in an appropriate format for input into the LDO script.

In [12]:
serialize("A_for_LDO",
Dict(
    "A"=>newton_res["A_newton"][end] |> ju_to_py, # the LDO script requires A to be PyObject. 
    "recursion_depth"=>newton_res["recursion_depth"]
))

Now we can run the LDO script. By default it will compute $10$ eigenvalues of the Gilt-TNR LDO.

In [32]:
chi_compression=20
run(`julia --project Lab/LDO_spectra.jl --rotate true --path_to_tensor "A_for_LDO" --chi_compression $chi_compression`)

The latest version of Julia in the `release` channel is 1.10.5+0.x64.linux.gnu. You currently have `1.10.4+0.x64.linux.gnu` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.5+0.x64.linux.gnu and update the `release` channel to that version.
/home/ebel/Codes/tmp2/GILT_TNR_R/GiltTNR/tensors/tensorcommon.py:255: UserWarning: dirs in dot do not match.
  warnings.warn("dirs in dot do not match.")
/home/ebel/Codes/tmp2/GILT_TNR_R/GiltTNR/tensors/tensorcommon.py:255: UserWarning: dirs in dot do not match.
  warnings.warn("dirs in dot do not match.")
/home/ebel/Codes/tmp2/GILT_TNR_R/GiltTNR/tensors/tensorcommon.py:255: UserWarning: dirs in dot do not match.
  warnings.warn("dirs in dot do not match.")
/home/ebel/Codes/tmp2/GILT_TNR_R/GiltTNR/tensors/tensorcommon.py:255: UserWarning: dirs in dot do not match.
  warnings.warn("dirs in dot do not match.")
/home/ebel/Codes/tmp2/GILT_TNR_R/GiltTNR/tensors/tensorcommon.py:255: UserWarning: dirs in dot do not match.
  w

SCALING DIMENSIONS FROM THE LDO FOR chi_compression=20:
path to tensor: A_for_LDO
0.0004  1.0
0.1257  -0.917
1.0081  0.497
1.1345  0.0
1.1345  0.0
2.0171  -0.247
2.0174  -0.247
2.0198  0.0
2.0198  0.0
2.1308  0.228
2.1447  -0.226
2.1677  0.223
END OF LIST FOR chi_compression=20:


Process(`julia --project Lab/LDO_spectra.jl --rotate true --path_to_tensor A_for_LDO --chi_compression 20`, ProcessExited(0))

We load the result and compute the scaling dimesnions. We multiply the scaling dimensions of $\mathbb{Z}_2$ odd operators by $-1$ if the eigenvalue for $\sigma$ is negative. 

In [33]:
vals=deserialize("LDO/rotate=true_30_6.0e-6_1.0e-10_rotate=(true)_path=A_for_LDO.data")["eigensystem"][1];

vecs=deserialize("LDO/rotate=true_30_6.0e-6_1.0e-10_rotate=(true)_path=A_for_LDO.data")["eigensystem"][2];

tmp=vecs[1][1,1,1,:]
dim_even=tmp[abs.(tmp).>1e-10] |> length

Z_for_compressed_leg=diagm(vcat(ones(dim_even),-ones(chi_compression-dim_even)))

parities=zeros(length(vecs))
for i ∈ eachindex(parities)
    parities[i]=parity(real.(vecs[i]),Z_for_compressed_leg)
end

if real.(vals[2])<0
    for i in eachindex(vals)
        if parities[i]<0
            vals[i]*=-1
        end
    end
end

dims=-log.(2,abs.(vals))
spins_mod_4=imag.(log.(sign.(vals))*2/π)


println("SCALING DIMESNIONS:")
dims.|>println;

println("SPINS MOD 4:")
spins_mod_4.|>println;


SCALING DIMESNIONS:
0.00043110310878929377
0.12572413275530778
1.008076725733081
1.1345003800844213
1.1345003800844213
2.0170727383064713
2.017402102433309
2.019817804416084
2.019817804416084
2.1308397257087845
2.1446830556023393
2.167739958308346
SPINS MOD 4:
0.0
-0.0
0.0
-1.0001127475470657
1.0001127475470657
2.0
2.0
0.9999820253317327
-0.9999820253317327
-2.0
-0.0
-2.0
